In [ ]:
import tensorflow as tf
from   tensorflow import keras
from   tensorflow.keras import regularizers
from   tensorflow.keras import Sequential
from   tensorflow.keras.layers import Dropout, Dense

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [ ]:
import config
import datasets
import models
import trade_processing as tp
import utils

In [ ]:
TICKER   = 'SPY'
WIN_PROB = 10
MARGIN   = 5
PROFIT   = 1

In [ ]:
options_model = models.load_best_model(TICKER, MARGIN, PROFIT)
percentiles = options_model.percentiles

In [ ]:
# Load the trades
trades_df = datasets.load_dataset(
    ticker=TICKER,
    max_margin=MARGIN,
    min_profit=PROFIT,
    total_datapoints=200000,
    verbose=True
).data

In [ ]:
unique_strats = trades_df.description.unique()
for s in unique_strats:
    s_bets = trades_df[trades_df.description == s]
    s_total = s_bets.shape[0]
    print('{:>25}: {} ({:.1%})'.format(s, s_total, s_total / trades_df.shape[0]))

In [ ]:
options_model.insert_predictions(trades_df)
trades_df.sort_values(by=['confidence'], ascending=False, inplace=True)
trades_df.head(20)

In [7]:
def print_details(strat_substrings=None, percentile=None):

    def formatter(desc, form, val):
        try:
            print('{:<25}: {}'.format(desc,form.format(val)))
        except TypeError:
            print('{:<25}: {}'.format(desc,form.format(*val)))

    df = trades_df
    if strat_substrings is not None:
        df = df[df.description.map(lambda d: all(s in d for s in strat_substrings))]
    if percentile is not None:
        thresh = percentiles[str(percentile)]
        formatter('Threshold', '{:.1f}', thresh)
        df = df[df.confidence >= thresh]

    count = df.shape[0]
    winners = df[(df.max_profit > PROFIT)]
    
    formatter('Total good bets',
              '{:<6} ({:>5.1%})', [count, count / trades_df.shape[0]])
    formatter('Total winners', '{:.2%}', winners.shape[0] / count)
    formatter('Median days to expiry',
              '{:.1f}', winners.seconds_to_expiry.median() / (60 * 60 * 24))
    print()
    profit = df.max_profit
    formatter('Min profit', '{:.2f}', profit.min())
    formatter('Median profit', '{:.2f}', profit.median())
    formatter('Max profit', '{:.2f}', profit.max())
    
    print()

    df.hist(
        column=['open_margin', 'max_profit'],
        sharey=True,
        bins=40,
    )
    plt.show()

In [ ]:
print_details(percentile=90)